In [46]:
import pandas as pd
import os

file_path = r"C:\Users\Deshan\Documents\IIT LECS\Year 2 Sem 1\DSGP\dataset creation\raw_dataset.csv"

df = pd.read_csv(file_path)

print("Columns in the dataset:")
print(df.columns.tolist())

print(df.info())


Columns in the dataset:
['GLOBAL UNIQUE IDENTIFIER', 'LAST EDITED DATE', 'TAXONOMIC ORDER', 'CATEGORY', 'TAXON CONCEPT ID', 'COMMON NAME', 'SCIENTIFIC NAME', 'SUBSPECIES COMMON NAME', 'SUBSPECIES SCIENTIFIC NAME', 'EXOTIC CODE', 'OBSERVATION COUNT', 'BREEDING CODE', 'BREEDING CATEGORY', 'BEHAVIOR CODE', 'AGE/SEX', 'COUNTRY', 'COUNTRY CODE', 'STATE', 'STATE CODE', 'COUNTY', 'COUNTY CODE', 'IBA CODE', 'BCR CODE', 'USFWS CODE', 'ATLAS BLOCK', 'LOCALITY', 'LOCALITY ID', 'LOCALITY TYPE', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE', 'TIME OBSERVATIONS STARTED', 'OBSERVER ID', 'SAMPLING EVENT IDENTIFIER', 'PROTOCOL TYPE', 'PROTOCOL CODE', 'PROJECT CODE', 'DURATION MINUTES', 'EFFORT DISTANCE KM', 'EFFORT AREA HA', 'NUMBER OBSERVERS', 'ALL SPECIES REPORTED', 'GROUP IDENTIFIER', 'HAS MEDIA', 'APPROVED', 'REVIEWED', 'REASON', 'TRIP COMMENTS', 'SPECIES COMMENTS', 'Unnamed: 49']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484 entries, 0 to 483
Data columns (total 50 columns):
 #   Column     

Missing values

In [47]:
missing_values = df.isnull().sum()
print(missing_values)


GLOBAL UNIQUE IDENTIFIER        0
LAST EDITED DATE                0
TAXONOMIC ORDER                 0
CATEGORY                        0
TAXON CONCEPT ID              484
COMMON NAME                     0
SCIENTIFIC NAME                 0
SUBSPECIES COMMON NAME        484
SUBSPECIES SCIENTIFIC NAME    484
EXOTIC CODE                   484
OBSERVATION COUNT               0
BREEDING CODE                 484
BREEDING CATEGORY             484
BEHAVIOR CODE                 484
AGE/SEX                       484
COUNTRY                         0
COUNTRY CODE                    0
STATE                           0
STATE CODE                      0
COUNTY                          0
COUNTY CODE                     0
IBA CODE                      371
BCR CODE                      484
USFWS CODE                    484
ATLAS BLOCK                   484
LOCALITY                        0
LOCALITY ID                     0
LOCALITY TYPE                   0
LATITUDE                        0
LONGITUDE     

In [48]:
relevant_columns = [
    'OBSERVATION DATE', 'TIME OBSERVATIONS STARTED', 'LATITUDE', 'LONGITUDE',
    'OBSERVATION COUNT', 'DURATION MINUTES', 'EFFORT DISTANCE KM',
    'NUMBER OBSERVERS'
]
#data_frame
df_relevant = df[relevant_columns]
print(df_relevant.head())


  OBSERVATION DATE TIME OBSERVATIONS STARTED  LATITUDE  LONGITUDE  \
0       2022-01-02                  16:30:00  6.079940  80.932975   
1       2022-01-03                  06:30:00  6.076015  80.932945   
2       2022-02-04                  15:42:00  6.083702  80.932344   
3       2022-02-04                  15:28:00  6.096865  80.924300   
4       2022-02-04                  15:28:00  6.096865  80.924300   

  OBSERVATION COUNT  DURATION MINUTES  EFFORT DISTANCE KM  NUMBER OBSERVERS  
0                 4             120.0                1.50               7.0  
1                 4              30.0                0.50               2.0  
2                 8              39.0                1.30               1.0  
3                 3               8.0                2.29               5.0  
4                 3               8.0                2.29               5.0  


In [51]:
df_relevant = df_relevant.copy()

df_relevant['OBSERVATION DATE'] = pd.to_datetime(df_relevant['OBSERVATION DATE'], errors='coerce')


df_relevant['Year'] = df_relevant['OBSERVATION DATE'].dt.year
df_relevant['Month'] = df_relevant['OBSERVATION DATE'].dt.month
df_relevant['DayOfWeek'] = df_relevant['OBSERVATION DATE'].dt.dayofweek

print(df_relevant.head())


  OBSERVATION DATE TIME OBSERVATIONS STARTED  LATITUDE  LONGITUDE  \
0       2022-01-02                  16:30:00  6.079940  80.932975   
1       2022-01-03                  06:30:00  6.076015  80.932945   
2       2022-02-04                  15:42:00  6.083702  80.932344   
3       2022-02-04                  15:28:00  6.096865  80.924300   
4       2022-02-04                  15:28:00  6.096865  80.924300   

  OBSERVATION COUNT  DURATION MINUTES  EFFORT DISTANCE KM  NUMBER OBSERVERS  \
0                 4             120.0                1.50               7.0   
1                 4              30.0                0.50               2.0   
2                 8              39.0                1.30               1.0   
3                 3               8.0                2.29               5.0   
4                 3               8.0                2.29               5.0   

   Year  Month  DayOfWeek  
0  2022      1          6  
1  2022      1          0  
2  2022      2          4 

In [53]:
# Drop rows with missing critical values
critical_columns = ['OBSERVATION DATE', 'LATITUDE', 'LONGITUDE', 'OBSERVATION COUNT']
df_cleaned = df.dropna(subset=critical_columns)



df_cleaned['OBSERVATION DATE'] = pd.to_datetime(df_cleaned['OBSERVATION DATE'], errors='coerce')

df_cleaned['TIME OBSERVATIONS STARTED'] = pd.to_datetime(df_cleaned['TIME OBSERVATIONS STARTED'], format='%H:%M:%S', errors='coerce')

df_cleaned['EFFORT DISTANCE KM'] = pd.to_numeric(df_cleaned['EFFORT DISTANCE KM'], errors='coerce')

df_cleaned['DURATION MINUTES'] = pd.to_numeric(df_cleaned['DURATION MINUTES'], errors='coerce')


df_cleaned['OBSERVATION COUNT'] = pd.to_numeric(df_cleaned['OBSERVATION COUNT'], errors='coerce')

# Extract Year, Month, and DayOfWeek from 'OBSERVATION DATE'
df_cleaned['Year'] = df_cleaned['OBSERVATION DATE'].dt.year
df_cleaned['Month'] = df_cleaned['OBSERVATION DATE'].dt.month
df_cleaned['DayOfWeek'] = df_cleaned['OBSERVATION DATE'].dt.dayofweek

# Define seasons for Sri Lanka
def get_season(month):
    if month in [12, 1, 2]:
        return 'Northeast Monsoon'
    elif month in [3, 4]:
        return 'First Inter-monsoon'
    elif month in [5, 6, 7]:
        return 'Southwest Monsoon'
    elif month in [8, 9]:
        return 'Second Inter-monsoon'
    else:
        return 'Unknown'

df_cleaned['Season'] = df_cleaned['Month'].apply(get_season)

# Extract Hour and Time of Day
df_cleaned['Hour'] = pd.to_datetime(df_cleaned['TIME OBSERVATIONS STARTED'], errors='coerce').dt.hour

def get_time_of_day(hour):
    if pd.isnull(hour):
        return None
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df_cleaned['TimeOfDay'] = df_cleaned['Hour'].apply(get_time_of_day)

# Normalize observation counts
df_cleaned['DURATION MINUTES'] = df_cleaned['DURATION MINUTES'].fillna(1)  # Fill NaN with 1 to avoid division by zero
df_cleaned['EFFORT DISTANCE KM'] = df_cleaned['EFFORT DISTANCE KM'].fillna(1)  # Fill NaN with 1

df_cleaned['CountPerMinute'] = df_cleaned['OBSERVATION COUNT'] / df_cleaned['DURATION MINUTES']
df_cleaned['CountPerKM'] = df_cleaned['OBSERVATION COUNT'] / df_cleaned['EFFORT DISTANCE KM']

# Display the cleaned DataFrame
print(df_cleaned.head())



                          GLOBAL UNIQUE IDENTIFIER  \
0  URN:CornellLabOfOrnithology:EBIRD:OBS1309546250   
1  URN:CornellLabOfOrnithology:EBIRD:OBS1309539456   
2  URN:CornellLabOfOrnithology:EBIRD:OBS1334048727   
3  URN:CornellLabOfOrnithology:EBIRD:OBS1334049465   
4  URN:CornellLabOfOrnithology:EBIRD:OBS1333992668   

             LAST EDITED DATE  TAXONOMIC ORDER CATEGORY  TAXON CONCEPT ID  \
0  2023-10-23 19:22:46.607435            24523  species               NaN   
1  2023-10-17 03:09:42.732041            24523  species               NaN   
2  2023-10-23 19:23:29.964364            24523  species               NaN   
3  2023-10-22 02:59:01.388532            24523  species               NaN   
4  2023-10-22 02:59:00.197386            24523  species               NaN   

         COMMON NAME   SCIENTIFIC NAME  SUBSPECIES COMMON NAME  \
0  Red-vented Bulbul  Pycnonotus cafer                     NaN   
1  Red-vented Bulbul  Pycnonotus cafer                     NaN   
2  Red-vented 

transforming

In [84]:
from sklearn.preprocessing import StandardScaler

# Scale numerical features
scaler = StandardScaler()
df_cleaned['ScaledCountPerMinute'] = scaler.fit_transform(df_cleaned[['CountPerMinute']].fillna(0))

# One-hot encode categorical features
df_encoded = pd.get_dummies(df_cleaned, columns=['Season', 'TimeOfDay'], drop_first=True)

print("\nTransformed dataset:")
print(df_encoded.head())



Transformed dataset:
                          GLOBAL UNIQUE IDENTIFIER  \
0  URN:CornellLabOfOrnithology:EBIRD:OBS1309546250   
1  URN:CornellLabOfOrnithology:EBIRD:OBS1309539456   
2  URN:CornellLabOfOrnithology:EBIRD:OBS1334048727   
3  URN:CornellLabOfOrnithology:EBIRD:OBS1334049465   
4  URN:CornellLabOfOrnithology:EBIRD:OBS1333992668   

             LAST EDITED DATE  TAXONOMIC ORDER CATEGORY  TAXON CONCEPT ID  \
0  2023-10-23 19:22:46.607435            24523  species               NaN   
1  2023-10-17 03:09:42.732041            24523  species               NaN   
2  2023-10-23 19:23:29.964364            24523  species               NaN   
3  2023-10-22 02:59:01.388532            24523  species               NaN   
4  2023-10-22 02:59:00.197386            24523  species               NaN   

         COMMON NAME   SCIENTIFIC NAME  SUBSPECIES COMMON NAME  \
0  Red-vented Bulbul  Pycnonotus cafer                     NaN   
1  Red-vented Bulbul  Pycnonotus cafer                    

In [85]:
from sklearn.model_selection import train_test_split

features = ['ScaledCountPerMinute', 'LATITUDE', 'LONGITUDE', 'Season_Northeast Monsoon', 'TimeOfDay_Morning']  # Corrected column names
X = df_encoded[features]
y = df_encoded['OBSERVATION COUNT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [86]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

y_train = np.array(y_train)
X_train = np.array(X_train)
X_test = np.array(X_test)

# Handle missing values
non_nan_mask = ~np.isnan(y_train)

X_train = X_train[non_nan_mask]
y_train = y_train[non_nan_mask]

imputer = SimpleImputer(strategy='mean')
y_train = imputer.fit_transform(y_train.reshape(-1, 1)).ravel()

# Initialize the model
model = RandomForestRegressor(random_state=42)

# Train the model
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [87]:
missing_percentage = np.isnan(y_train).mean() * 100
print(f"Percentage of missing values in y_train: {missing_percentage:.2f}%")

Percentage of missing values in y_train: 0.00%


In [88]:
import numpy as np
from sklearn.impute import SimpleImputer

if np.isnan(y_test).any():
   
    imputer = SimpleImputer(strategy='mean')
    
    y_test = imputer.fit_transform(y_test.values.reshape(-1, 1))
    y_test = y_test.ravel()  # Flatten back to a 1D array


y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nModel Performance:\nMean Squared Error: {mse}\nR2 Score: {r2}")


Model Performance:
Mean Squared Error: 25.926062064795985
R2 Score: 0.48346858090058487


In [83]:
import pandas as pd

# Use the already processed `df_encoded` from your workspace
# Filter for necessary columns
fields_to_keep = [
    'LATITUDE',
    'LONGITUDE',
    'ScaledCountPerMinute',
    'OBSERVATION COUNT'
]
fields_to_keep += [col for col in df_encoded.columns if col.startswith('Season_')]
fields_to_keep += [col for col in df_encoded.columns if col.startswith('TimeOfDay_')]

# Create the final dataset
df_final = df_encoded[fields_to_keep]

# Save the final dataset
output_file_path = r"C:\Users\Deshan\Documents\IIT LECS\Year 2 Sem 1\DSGP\dataset creation\final_dataset.csv"
df_final.to_csv(output_file_path, index=False)

# Print output confirmation
print(f"Final dataset saved to {output_file_path}")
print("Columns in the final dataset:")
print(df_final.columns.tolist())
print("\nPreview of the final dataset:")
print(df_final.head())


Final dataset saved to C:\Users\Deshan\Documents\IIT LECS\Year 2 Sem 1\DSGP\dataset creation\final_dataset.csv
Columns in the final dataset:
['LATITUDE', 'LONGITUDE', 'ScaledCountPerMinute', 'OBSERVATION COUNT', 'Season_Northeast Monsoon', 'Season_Second Inter-monsoon', 'Season_Southwest Monsoon', 'Season_Unknown', 'Season_Time', 'TimeOfDay_Evening', 'TimeOfDay_Morning']

Preview of the final dataset:
   LATITUDE  LONGITUDE  ScaledCountPerMinute  OBSERVATION COUNT  \
0  6.079940  80.932975             -0.074096                4.0   
1  6.076015  80.932945             -0.052122                4.0   
2  6.083702  80.932344             -0.036346                8.0   
3  6.096865  80.924300              0.000982                3.0   
4  6.096865  80.924300              0.000982                3.0   

   Season_Northeast Monsoon  Season_Second Inter-monsoon  \
0                      True                        False   
1                      True                        False   
2           